In [1]:
import sys
sys.path.append("../scripts/")
from mcl import *

Duplicate key in file WindowsPath('C:/Users/tauyu/anaconda/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 257 ('font.family:  IPAexGothic')


<IPython.core.display.Javascript object>

In [2]:
class ResetMcl(Mcl):
    def __init__(self,envmap,init_pose,num,motion_noise_stds={"nn":0.19,"no":0.001,"on":0.13,"oo":0.2},
                distance_dev_rate=0.14,direction_dev=0.05,alpha_threshold=0.001,expansion_rate=0.2):
        super().__init__(envmap,init_pose,num,motion_noise_stds,distance_dev_rate,direction_dev)
        self.alpha_threshold=alpha_threshold
        self.expansion_rate=expansion_rate
        self.counter=0
        
    def random_reset(self):
         for p in self.particles:
                p.pose=np.array([np.random.uniform(-5.0,5,0),np.random.uniform(-5.0,5.0),np.random.uniform(-math.pi,math.pi)]).T
                p.weight=1/len(self.particles)
    
    def sensor_resetting_draw(self,particle,landmark_pos,ell_obs,phi_obs):
             ##パーティクルの位置を決める##
                psi=np.random.uniform(-np.pi,np.pi) #ランドマークからの方角を選ぶ
                ell=norm(loc=ell_obs,scale=(ell_obs*self.distance_dev_rate)**2).rvs() #ランドマークからの距離を選ぶ
                particle.pose[0]=landmark_pos[0]+ell*math.cos(psi)
                particle.pose[1]=landmark_pos[1]+ell*math.sin(psi)
                
            ##パーティクルの向きを決める##
                phi=norm(loc=phi_obs,scale=(self.direction_dev)**2).rvs() #ランドマークが見える向きを決める
                particle.pose[2]=math.atan2(landmark_pos[1]-particle.pose[1],landmark_pos[0]-particle.pose[0])-phi
            
                particle.weight=1.0/len(self.particles) #重みを1/Nに
            
    def sensor_resetting(self,observation):
        nearest_obs = np.argmin([obs[0][0] for obs in observation]) #距離が一番近いランドマークを選択
        values, landmark_id = observation[nearest_obs]
        
        for p in self.particles:
            self.sensor_resetting_draw(p, self.map.landmarks[landmark_id].pos, *values)
            
            
    def expansion_resetting(self):
        for p in self.particles:
            p.pose+=multivariate_normal(cov=np.eye(3)*(self.expansion_rate**2)).rvs()
            p.weight=1.0/len(self.particles)
            
    def observation_update(self,observation):
        for p in self.particles:
            p.observation_update(observation,self.map,self.distance_dev_rate,self.direction_dev)
     
        self.set_ml()
        
        if sum([p.weight for p in self.particles]) < self.alpha_threshold:
            self.counter += 1
            if self.counter < 5:
                self.expansion_resetting() #expansion_resettingに変更
            else:
                self.sensor_resetting(observation)
        else:
            self.counter=0
            self.resampling()

In [3]:
def trial_kidnap_trap(animation): ###mclkidnap1test
    time_interval=0.1
    world=World(30,time_interval,debug=not animation)
    
    ##地図を生成して3つランドマークを追加##
    m=Map()
    m.append_landmark(Landmark(-4,2))
    m.append_landmark(Landmark(2,-3))
    m.append_landmark(Landmark(3,3))
    world.append(m)
    
    ##ロボットを作る##
    init_pose=np.array([np.random.uniform(-5.0,5.0),np.random.uniform(-5.0,5.0),np.random.uniform(-math.pi,math.pi)]).T
    robot_pose=np.array([np.random.uniform(-5.0,5.0),np.random.uniform(-5.0,5.0),np.random.uniform(-math.pi,math.pi)]).T
    pf=ResetMcl(m,init_pose,100)
    a=EstimationAgent(time_interval,0.2,10.0/180*math.pi,pf)
    r=Robot(robot_pose,sensor=Camera(m),agent=a,color="red",expected_stuck_time=5,expected_escape_time=5)
    world.append(r)
    
    world.draw()
    
    return(r.pose, pf.ml.pose)

In [4]:
trial_kidnap_trap(True)

C:\Users\tauyu\anaconda\lib\site-packages\matplotlib\animation.py:889: UserWarning: Animation was deleted without rendering anything. This is most likely not intended. To prevent deletion, assign the Animation to a variable, e.g. `anim`, that exists until you have outputted the Animation using `plt.show()` or `anim.save()`.
  warnings.warn(


<IPython.core.display.Javascript object>

(array([ 1.30181115, -3.54503453,  1.14658102]),
 array([-4.35185836, -3.05564646,  1.9251807 ]))

In [5]:
if __name__=='__main__': ###mclglobal1exec
    ok=0
    for i in range(1000):
        actual, estm=trial_kidnap_trap(False)
        diff = math.sqrt((actual[0]-estm[0])**2 + (actual[1]-estm[1])**2)
        print(i, "真値:",actual, "推定値:", estm, "誤差:", diff)
        if diff<=1.0:
            ok += 1
            print(ok)

0 真値: [-6.14579147  4.38077029  5.49144458] 推定値: [-2.0771988   0.0962599   2.34174837] 誤差: 5.908508746774762
1 真値: [1.47048815 3.66617954 3.46051455] 推定値: [1.17681241 0.77339606 2.93462792] 誤差: 2.9076522652208956
2 真値: [-4.29244334 -2.70172226  4.00831179] 推定値: [-0.57312829  3.06207134 -0.74011722] 誤差: 6.8596371017528215
3 真値: [-5.86240843 -6.22235736  5.7766791 ] 推定値: [-2.2746049   4.34251996  5.00602392] 誤差: 11.157462389669279
4 真値: [ 4.31434418 -3.48559324 -0.22276634] 推定値: [ 0.89472542 -0.0547556   3.81380011] 誤差: 4.844010672337221
5 真値: [4.73475404 5.59726063 1.8316016 ] 推定値: [0.0098241  1.23076015 7.41508371] 誤差: 6.4336062589779806
6 真値: [ 1.607341   -5.35584549 -0.33722416] 推定値: [3.07171564 1.45652554 2.31159079] 誤差: 6.967983367442867
7 真値: [ 0.18799571 -3.41919154  1.49384887] 推定値: [ 5.0458329  -1.33227247  7.09918544] 誤差: 5.287136592396117
8 真値: [-1.73367543 -4.3394624   6.19389289] 推定値: [-3.09118414  2.08661944  5.04994135] 誤差: 6.567903600321683
9 真値: [-2.1238989   5.62235273

C:\Users\tauyu\section_advanced_localization\../scripts\ideal_robot.py:30: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(4,4))


20 真値: [ 2.25157608 -3.50441831  4.48790401] 推定値: [ 2.43799378 -3.31190206 -1.78744816] 誤差: 0.2679814667435564
2
21 真値: [ 4.43668868 -3.85886366  0.66174363] 推定値: [ 0.24712717 -5.11672757  6.79908801] 誤差: 4.374316773531818
22 真値: [-3.89266968  2.06893392  4.07635586] 推定値: [-4.17514015  1.94758785  3.71429825] 誤差: 0.3074320025781304
3
23 真値: [0.41880055 1.14591603 3.17084403] 推定値: [-0.45029164  2.62473916  3.41419355] 誤差: 1.715295618097203
24 真値: [0.03459692 6.49063496 2.50529201] 推定値: [-0.06168889 -1.47698325  1.85417765] 誤差: 7.968199982595877
25 真値: [-2.60364047  2.18370646  5.85236107] 推定値: [ 4.47318462  8.19979172 -2.4758449 ] 誤差: 9.288419416757467
26 真値: [ 1.38070618 -1.55754606  1.69822792] 推定値: [ 3.99484535 -0.45886078  2.30052317] 誤差: 2.835636247621197
27 真値: [ 4.37242547 -7.03903718  5.97934038] 推定値: [-0.55428447  0.09478703  4.72165605] 誤差: 8.669712722250328
28 真値: [-4.3865965  -2.55908497  3.70306946] 推定値: [-4.48542419 -3.68336853  5.12161671] 誤差: 1.1286188222363678
29 真値: [-